In [39]:
import heapq
import random

class Cell:
    def __init__(self, x, y, cost):
        self.x = x
        self.y = y
        self.cost = cost
        self.parent = None

    def __lt__(self, other): 
        return self.cost < other.cost

class Forest:

    def __init__(self, size, start=(0, 0), goal=(-1,-1), obs_rate=0.1):
        self.size = size
        self.grid = [[ -1 for _ in range(size)] for _ in range(size)] 
        
        self.start = start

        if goal == (-1, -1):
            self.goal = (size - 1, size - 1)
        else:
            self.goal = goal


        self.grid[self.start[0]][self.start[1]] = -1  
        self.grid[self.goal[0]][self.goal[1]] = -1    

        self.mark_cells(obs_rate, -2) # obsticle

  
    def mark_cells(self, rate, symbol): 
        num_cells = self.size * self.size - 2

        num_chosen_cells = int(num_cells * rate) 

        indices = [(i, j) for i in range(self.size) for j in range(self.size)
                   if not (i == 0 and j == 0) and not (i == self.size - 1 and j == self.size - 1)] 

        chosen_indices = random.sample(indices, num_chosen_cells) 

        for i, j in chosen_indices:
            self.grid[i][j] = symbol 

    
    def is_valid_move(self, x, y): 
        return 0 <= x < self.size and 0 <= y < self.size

   
    def get_neighbors_nodes(self, node): 

        neighbors = []
        # directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
        directions = [(1, 0), (0, 1), (1, 1)]

        for dx, dy in directions:
            nx, ny = node.x + dx, node.y + dy

            if self.is_valid_move(nx, ny):
                neighbors.append(Cell(nx, ny, self.grid[nx][ny]))

        return neighbors
     

    def bf_search(self):
        start_node = Cell(self.start[0], self.start[1], 0)
        queue = [ (self.heuristic_cost(start_node), start_node) ]
        explored = set() 

        while queue:
            current_cost, current_node = heapq.heappop(queue) 

            if (current_node.x, current_node.y) == self.goal: 
                print('Total Cost:', current_cost)
                print(current_cost)
                return self.construct_path(current_node)

            explored.add((current_node.x, current_node.y))

            for neighbor in self.get_neighbors_nodes(current_node): 

                if current_node.cost == -2: 
                            continue

                new_cost = self.heuristic_cost(neighbor) + current_cost

                # if (current_node.x, current_node.y) != self.start:
                    # new_cost -= self.heuristic_cost(current_node)

                if (neighbor.x, neighbor.y) not in explored :
                    heapq.heappush(queue, (new_cost, neighbor))
                    neighbor.parent = current_node 


        print('Got Stuck')

    
    def calculate_cost(self, curr, neighbor):
        new_cost = 0

        if neighbor.x == curr.x + 1 and neighbor.x == curr.x or neighbor.x == curr.x and neighbor.x + 1 == curr.x:
            new_cost = 2
        else:
            new_cost = 3

        return new_cost


     
    def a_star_treasure_search(self):
        start_node = Cell(self.start[0], self.start[1], 0)
        queue = [(0, start_node)]
        explored = set() # Used to store all explored cells
        all_costs = {(self.start[0], self.start[1]): 0}

        while queue:
            current_cost, current_node = heapq.heappop(queue) 

            if (current_node.x, current_node.y) == self.goal: 
                print('Total Cost:', all_costs[(current_node.x, current_node.y)], end='\n\n')
                return self.construct_path(current_node), all_costs
       

            explored.add((current_node.x, current_node.y))

            for neighbor in self.get_neighbors_nodes(current_node):

                if current_node.cost == -2: 
                    continue 

                n_cost = self.calculate_cost(current_node, neighbor)

                if (neighbor.x, neighbor.y) not in all_costs:
                    all_costs[(neighbor.x, neighbor.y)] = all_costs[(current_node.x, current_node.y)] + n_cost
                
                elif (all_costs[(current_node.x, current_node.y)] + n_cost) < all_costs[(neighbor.x, neighbor.y)]:
                    all_costs[(neighbor.x, neighbor.y)] = all_costs[(current_node.x, current_node.y)] + n_cost

                
                new_cost = all_costs[(neighbor.x, neighbor.y)]  + self.heuristic_cost(neighbor)


                if (neighbor.x, neighbor.y) not in explored :
                    heapq.heappush(queue, (new_cost, neighbor))
                    neighbor.parent = current_node  


        print('Got Stuck')
    


    '''
    def a_star_treasure_search(self):
        start_node = Cell(self.start[0], self.start[1], 0)
        queue = [(0, start_node)]
        explored = set() 

        while queue:
            current_cost, current_node = heapq.heappop(queue) 

            if (current_node.x, current_node.y) == self.goal: 
                print('Total Cost:', current_cost-self.heuristic_cost(current_node))
                return self.construct_path(current_node)
            

            explored.add((current_node.x, current_node.y))

            for neighbor in self.get_neighbors_nodes(current_node): 

                if current_node.cost == -2: 
                        continue

                n_cost = self.calculate_cost(current_node, neighbor)
                
                if (current_node.x, current_node.y) != self.start:
                    new_cost = current_cost + n_cost + self.heuristic_cost(neighbor) - self.heuristic_cost(current_node)
                else:
                    new_cost = current_cost + n_cost + self.heuristic_cost(neighbor) 


                if (neighbor.x, neighbor.y) not in explored :
                    heapq.heappush(queue, (new_cost, neighbor))
                    neighbor.parent = current_node 



        print('Got Stuck')
    '''

    def heuristic_cost(self, state):
        return ( abs(self.goal[0] - state.x) + abs(self.goal[1] - state.y) )

    
    def construct_path(self, node): 
        path = []
        while node:
            path.append((node.x, node.y))
            node = node.parent
        return path[::-1]

   
    def display(self, path): 

        print('\nOriginal Maze:')
        for i in range(self.size):
            for j in range(self.size):
                if (i, j) == self.start:
                    print('S', end=' ')
                elif (i, j) == self.goal:
                    print('G', end=' ')
                elif self.grid[i][j] == -1:
                    print('0', end=' ')
                elif self.grid[i][j] == -2:
                    print('-', end=' ')
                else:
                    print(self.grid[i][j], end=' ')
            print()

        print()
        print()
        print()

        print('Traversed Maze:')
        for i in range(self.size):
            for j in range(self.size):
                if (i, j) == self.start:
                    print('S', end=' ')
                elif (i, j) == self.goal:
                    print('G', end=' ')
                elif path != None and (i, j) in path:
                    print('*', end=' ')
                elif self.grid[i][j] == -1:
                    print('0', end=' ')
                elif self.grid[i][j] == -2:
                    print('-', end=' ')
                else:
                    print(self.grid[i][j], end=' ')
            print()




forest = Forest(15, obs_rate=0.0)
# path = forest.bf_search()
path, costs = forest.a_star_treasure_search()
print("Optimal Path:")
print(path)

if path != None:
    print('Cost per Step:')
    for key in costs:
        if key in path:
            print('{}:{}'.format(key, costs[key]), end=' , ')

print("\nEnchanted Forest Grid:")
forest.display(path)



Total Cost: 21

Optimal Path:
[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7)]
Cost per Step:
(0, 0):0 , (1, 1):3 , (2, 2):6 , (3, 3):9 , (4, 4):12 , (5, 5):15 , (6, 6):18 , (7, 7):21 , 
Enchanted Forest Grid:

Original Maze:
S 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 G 



Traversed Maze:
S 0 0 0 0 0 0 0 
0 * 0 0 0 0 0 0 
0 0 * 0 0 0 0 0 
0 0 0 * 0 0 0 0 
0 0 0 0 * 0 0 0 
0 0 0 0 0 * 0 0 
0 0 0 0 0 0 * 0 
0 0 0 0 0 0 0 G 
